In [4]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.datastore import Datastore
from azureml.core.dataset import Dataset

from sklearn import datasets

In [5]:
ws = Workspace.from_config()

In [6]:
datastores = ws.datastores

In [7]:
datastore_name = 'edvanstorage__azureml'
dataset_path = 'boston_houses/'
datastore = Datastore.get(ws, datastore_name=datastore_name)

In [8]:
boston = datasets.load_boston()

In [9]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [10]:
bos = pd.DataFrame(boston.data)
bos.columns = ['CrimeRate', 'ResidentialZoning', 'IndustrialZoning', 'OnRiver', 'NOXConcentration', 'NumberOfRooms', 'PreWarHouses', 'DistanceToEmployment', 'DistanceToHighways','PropertyTaxRate', 'ParentTeachRatio', 'ProportionAA', 'LowerStatusProportion']
bos['Price'] = boston.target
bos.head(5)

,CrimeRate,ResidentialZoning,IndustrialZoning,OnRiver,NOXConcentration,NumberOfRooms,PreWarHouses,DistanceToEmployment,DistanceToHighways,PropertyTaxRate,ParentTeachRatio,ProportionAA,LowerStatusProportion,Price
0,0.01,18.00,2.31,0.00,0.54,6.58,65.20,4.09,1.00,296.00,15.30,396.90,4.98,24.00
1,0.03,0.00,7.07,0.00,0.47,6.42,78.90,4.97,2.00,242.00,17.80,396.90,9.14,21.60
2,0.03,0.00,7.07,0.00,0.47,7.18,61.10,4.97,2.00,242.00,17.80,392.83,4.03,34.70
3,0.03,0.00,2.18,0.00,0.46,7.00,45.80,6.06,3.00,222.00,18.70,394.63,2.94,33.40
4,0.07,0.00,2.18,0.00,0.46,7.15,54.20,6.06,3.00,222.00,18.70,396.90,5.33,36.20


In [23]:
local_path = 'data/boston_data.csv'
bos.to_csv(path_or_buf=local_path)

In [24]:
datastore.upload(src_dir='data',
                 target_path=dataset_path,
                 overwrite=True,
                 show_progress=True)

Uploading an estimated of 1 files
Uploading data/boston_data.csv
Uploaded data/boston_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_60c0892f1c864efcbf19dacdcaa5f97e

## Upgrade AzureML


In [29]:
import sys
!{sys.executable} -m pip install azureml-sdk[automl] -U
!{sys.executable} -m pip install azureml-dataprep[pandas] -U

!{sys.executable} -m pip install pyarrow -U

Requirement already up-to-date: azureml-sdk[automl] in /data/anaconda/envs/py36/lib/python3.6/site-packages (1.0.57)


Requirement already up-to-date: azureml-dataprep[pandas] in /data/anaconda/envs/py36/lib/python3.6/site-packages (1.1.12)
     |████████████████████████████████| 58.1MB 46.5MB/s eta 0:00:01
  Found existing installation: pyarrow 0.11.1
    Uninstalling pyarrow-0.11.1:
      Successfully uninstalled pyarrow-0.11.1


In [27]:
!jupyter kernelspec list

Available kernels:
  ir                 /usr/local/share/jupyter/kernels/ir
  julia-0.6          /usr/local/share/jupyter/kernels/julia-0.6
  pyspark3kernel     /usr/local/share/jupyter/kernels/pyspark3kernel
  pysparkkernel      /usr/local/share/jupyter/kernels/pysparkkernel
  python2            /usr/local/share/jupyter/kernels/python2
  python3            /usr/local/share/jupyter/kernels/python3
  python3-azureml    /usr/local/share/jupyter/kernels/python3-azureml
  python3-mls        /usr/local/share/jupyter/kernels/python3-mls
  spark-3-python     /usr/local/share/jupyter/kernels/spark-3-python
  spark-python       /usr/local/share/jupyter/kernels/spark-python
  sparkkernel        /usr/local/share/jupyter/kernels/sparkkernel
  sparkrkernel       /usr/local/share/jupyter/kernels/sparkrkernel


In [12]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "cpucluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", 
                                                                vm_priority = 'lowpriority',
                                                                min_nodes = 1,
                                                                max_nodes = 100)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)


print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


TypeError: refresh_state() takes 1 positional argument but 2 were given

In [15]:
# state = compute_target.get_status()

In [37]:
new_min = 20
compute_target.update(min_nodes=new_min)
compute_target.refresh_state()
compute_target.wait_for_completion(show_output = True, min_node_count = new_min, timeout_in_minutes = 20)

UpdatingWaiting for cluster to scale. 1 out of 20 nodes ready.

SucceededWaiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1 out of 20 nodes ready.
.Waiting for cluster to scale. 1

In [38]:
new_min = 0
compute_target.update(min_nodes=new_min)
compute_target.refresh_state()
compute_target.wait_for_completion(show_output = True, min_node_count = new_min, timeout_in_minutes = 20)

Updating
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
